# O2O优惠券线下使用情况

https://blog.csdn.net/BiLiBa123/article/details/128020313?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522168272786316800184144333%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=168272786316800184144333&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-128020313-null-null.142^v86^control,239^v2^insert_chatgpt&utm_term=%E4%BC%98%E6%83%A0%E5%88%B8%E4%BD%BF%E7%94%A8%E6%83%85%E5%86%B5%E5%88%86%E6%9E%90&spm=1018.2226.3001.4187

数据清洗工具：jupyter notebook

数据分析工具：Python

可视化图表工具：Pyecharts

分析类型：描述性分析，诊断分析

分析方法：漏斗分析，RFM用户价值分析，AARRR分析，假设检验分析。

## 一.数据清洗

        1.1数据集来源

数据来源：https://tianchi.aliyun.com/competition/entrance/231593/information

        1.2数据说明
数据说明：用户在2016年1月1日至2016年6月30日之间真实线下消费行为数据

分析工具：Python

Table 1: 用户线下消费和优惠券领取行为

| Field | Description |  
| :--- | :--- |
| User_id | 用户ID |  
| Merchant_id | 商户ID|  
|Coupon_id | 优惠券ID：null表示无优惠券消费，此时Discount_rate和Date_received字段无意义 |
| Discount_rate | 优惠率：x \in [0,1]代表折扣率；x:y表示满x减y。单位是元 |
| Distanceuser | 经常活动的地点离该merchant的最近门店距离是x*500米（如果是连锁店，则取最近的一家门店），x\in[0,10]；null表示无此信息，0表示低于500米，10表示大于5公里；|
| Date_received | 领取优惠券日期 |
| Date | 消费日期：如果Date=null & Coupon_id != null，该记录表示领取优惠券但没有使用，即负样本；如果Date!=null & Coupon_id = null，则表示普通消费日期；如果Date!=null & Coupon_id != null，则表示用优惠券消费日期，即正样本； |

        1.3数据导入

        1.4查看缺失值

        1.5 数据处理​编辑

        1.6 保存文件​编辑

## 二.将文件导入用Python分析

        2.1查询各列数据为空值的数量 

## 三.总体情况统计分析

        3.1 数据总体情况 

        3.2 优惠券每月核销情况统计

        3.3 每月新增用户数

## 四.用户分析
        4.1 获客：每日新增用户情况

        4.2 转化及流失情况

        4.3 用户行为转化漏斗计算

        4.4 用户留存情况

        4.5 用户价值分析  
        
            4.5.1 先查询用户最近的消费时间间隔及消费频次

            4.5.2  分别查询R,F的最大值，最小值以及各值数量分布情况

            4.5.3 根据R,F值最大值和最小值的区间设计本次的打分标准，计算R,F的值

            4.5.4 查询R,F值的平均值

            4.5.5 根据平均值和用户分类规则表对用户分类

            4.5.6 查询各分层用户数量

## 五.商户分析

        5.1 每月有消费商户统计

        5.2 一周内每天有消费的商户数量

        5.3 商户整体核销率，核销率在0.25以下的商户占比

        5.4 商户分层

## 六.优惠券分析

        6.1 优惠券每日领取，每日核销情况

        6.2 优惠券平均核销时间

        6.3 不同距离下优惠券的领取量及核销率

## 七. 相关分析
        7.1 门店距离与优惠券核销率的相关性

        7.2 优惠券折扣与核销率的相关性

## 八.Tableau可视化图表

## 九.数据分析